In [12]:
#@title Install dependence

from IPython.utils import io
import os
import subprocess
import tqdm.notebook

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

try:
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      # Uninstall default Colab version of TF.
      %shell pip uninstall -y tensorflow
      # install directly from github
      %shell rm -rf ./deepmind-research
      pbar.update(1)
      %shell git clone https://github.com/deepmind/deepmind-research.git
      pbar.update(60)
      %shell cd deepmind-research/density_functional_approximation_dm21 && pip install .
      pbar.update(39)

except subprocess.CalledProcessError:
  print(captured)
  raise

  0%|          | 0/100 [elapsed: 00:00 remaining: ?]

In [15]:
#@title import modules
import density_functional_approximation_dm21 as dm21
from pyscf import gto
from pyscf import dft
from timeit import default_timer as timer

In [21]:
#@title Atomization energy of CH4
#@markdown build molecules

start = timer()
# Create the molecule of interest and select the basis set.
methane = gto.Mole()
methane.atom = """H 0.000000000000 0.000000000000 0.000000000000
                  C 0.000000000000 0.000000000000 1.087900000000
                  H 1.025681956337 0.000000000000 1.450533333333
                  H -0.512840978169 0.888266630391 1.450533333333
                  H -0.512840978169 -0.888266630391 1.450533333333"""
methane.basis = 'def2-qzvp'
# methane.verbose = 4
methane.build()

carbon = gto.Mole()
carbon.atom = 'C 0.0 0.0 0.0'
carbon.basis = 'def2-qzvp'
carbon.spin = 2
# carbon.verbose = 4
carbon.build()

hydrogen = gto.Mole()
hydrogen.atom = 'H 0.0 0.0 0.0'
hydrogen.basis = 'def2-qzvp'
hydrogen.spin = 1
# hydrogen.verbose = 4
hydrogen.build()

cp0 = timer()
print(f'Building Molecules: {cp0 - start:.2f}s')

Building Molecules: 0.57s


In [31]:
#@title DFT with DM21
#@markdown Running the SCF calculation with B3LYP initial guess and than DM21 for the energy

cp1 = timer()
energies = []
loop_timers = []
for mol in [methane, carbon, hydrogen]:
  # Create a DFT solver and insert the DM21 functional into the solver.
  if mol.spin == 0:
    mf = dft.RKS(mol)
  else:
    mf = dft.UKS(mol)
  # It will make SCF faster to start close to the solution with a cheaper
  # functional.
  mf.xc = 'B3LYP'
  mf.run()
  dm0 = mf.make_rdm1()
  cp_b3lyp = timer()

  mf._numint = dm21.NeuralNumInt(dm21.Functional.DM21)
  # It's wise to relax convergence tolerances.
  mf.conv_tol = 1E-6
  mf.conv_tol_grad = 1E-3
  # Run the DFT calculation.
  energy = mf.kernel(dm0=dm0)
  energies.append(energy)
  cp_dm21 = timer()
  loop_timers.append((cp_b3lyp, cp_dm21))
end1 = timer()
print({'CH4': energies[0], 'C': energies[1], 'H': energies[2]})
print(f'atomization energy = {(4*energies[2]+energies[1]-energies[0])*627.51}')
print(
f'''
CH4:
Run B3LYP: {loop_timers[0][0] - cp1:.2f}s
Run DM21: {loop_timers[0][1] - loop_timers[0][0]:.2f}s
C:
Run B3LYP: {loop_timers[1][0] - loop_timers[0][1]:.2f}s
Run DM21: {loop_timers[1][1] - loop_timers[1][0]:.2f}s
H:
Run B3LYP: {loop_timers[2][0] - loop_timers[1][1]:.2f}s
Run DM21: {loop_timers[2][1] - loop_timers[2][0]:.2f}s

Total: {end1 - cp1:.2f}s
''')

converged SCF energy = -40.5063134871814
converged SCF energy = -40.5178538431857
converged SCF energy = -37.8402038185573  <S^2> = 2.0037163  2S+1 = 3.0024765
converged SCF energy = -37.8454094823897  <S^2> = 2.0039033  2S+1 = 3.0026011
converged SCF energy = -0.499017789122656  <S^2> = 0.75  2S+1 = 2
converged SCF energy = -0.50115339206198  <S^2> = 0.75  2S+1 = 2
{'CH4': -40.51785384318569, 'C': -37.84540948238972, 'H': -0.5011533920619802}
atomization energy = 419.0705006318284

CH4:
Run B3LYP: 39.64s
Run DM21: 720.45s
C:
Run B3LYP: 2.36s
Run DM21: 52.73s
H:
Run B3LYP: 2.26s
Run DM21: 28.47s

Total: 845.92s



In [30]:
#@title Comparing with B3LYP
#@markdown Running the SCF calculation with B3LYP only

cp2 = timer()
loop_timers = []
energies = []
for mol in [methane, carbon, hydrogen]:
  if mol.spin == 0:
    mf = dft.RKS(mol)
  else:
    mf = dft.UKS(mol)
  mf.xc = 'B3LYP'
  mf.conv_tol = 1E-6
  mf.conv_tol_grad = 1E-3
  energy = mf.kernel()
  energies.append(energy)
  loop_timers.append(timer())

end2 = timer()
print({'CH4': energies[0], 'C': energies[1], 'H': energies[2]})
print(f'atomization energy = {(4*energies[2]+energies[1]-energies[0])*627.51}')
print(
f'''
CH4:
Run B3LYP: {loop_timers[0] - cp2:.2f}s
C:
Run B3LYP: {loop_timers[1] - loop_timers[0]:.2f}s
H:
Run B3LYP: {loop_timers[2] - loop_timers[1]:.2f}s

Total: {end2 - cp2:.2f}s
''')

converged SCF energy = -40.5063134871793
converged SCF energy = -37.8402038274101  <S^2> = 2.0037163  2S+1 = 3.0024765
converged SCF energy = -0.499017789122587  <S^2> = 0.75  2S+1 = 2
{'CH4': -40.506313487179305, 'C': -37.840203827410136, 'H': -0.49901778912258676}
atomization energy = 420.4558611924953

CH4:
Run B3LYP: 36.11s
C:
Run B3LYP: 1.78s
H:
Run B3LYP: 0.55s

Total: 38.45s



In [29]:
#@title Comparing with wB97X-D
#@markdown Running the SCF calculation with B3LYP only

cp2 = timer()
loop_timers = []
energies = []
for mol in [methane, carbon, hydrogen]:
  if mol.spin == 0:
    mf = dft.RKS(mol)
  else:
    mf = dft.UKS(mol)
  mf.xc = 'wB97XD'
  mf.conv_tol = 1E-6
  mf.conv_tol_grad = 1E-3
  energy = mf.kernel()
  energies.append(energy)
  loop_timers.append(timer())

end2 = timer()
print({'CH4': energies[0], 'C': energies[1], 'H': energies[2]})
print(f'atomization energy = {(4*energies[2]+energies[1]-energies[0])*627.51}')
print(
f'''
CH4:
Run wB97X-D: {loop_timers[0] - cp2:.2f}s
C:
Run wB97X-D: {loop_timers[1] - loop_timers[0]:.2f}s
H:
Run wB97X-D: {loop_timers[2] - loop_timers[1]:.2f}s

Total: {end2 - cp2:.2f}s
''')

converged SCF energy = -40.5234209614309
converged SCF energy = -37.8443082929958  <S^2> = 2.0052457  2S+1 = 3.0034951
converged SCF energy = -0.50287500652346  <S^2> = 0.75  2S+1 = 2
{'CH4': -40.52342096143088, 'C': -37.844308292995755, 'H': -0.5028750065234602}
atomization energy = 418.9336091955782

CH4:
Run wB97X-D: 63.56s
C:
Run wB97X-D: 2.53s
H:
Run wB97X-D: 0.72s

Total: 66.82s

